In [1]:
from importlib.metadata import version
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import numpy as np
import math
from dataclasses import dataclass
from matplotlib import pyplot as plt
import time
import os

# Pour torch si vous avez un GPU
# device = "cpu" if not torch.cuda.is_available() else "cuda"
device = "cpu" # Pour forcer l'utilisation du CPU

In [ ]:
# Environement potentielement testé
from environnement.environnement import Environnement as env # mother class
from environnement.environnement1 import Environnement1 as env1
from environnement.environnement2Str import Environnement2 as env2Str
from environnement.environnement3Str import Environnement3 as env3Str
from environnement.environnement6Str import Environnement6 as env6Str
from environnement.small_loop import small_loop

# model machine learning
from model.DeepNN import *
from model.Tokenizer import *
from outil import *
from inter.interactions import Interaction
from inter.simpleInteraction import simpleInteraction as inter

# L'agent qui choisit son destin
Dans se notebook nous reprennons la méthode de prédiciton de l'agent 1, mais cette fois l'action ne sera plus pris au hasard. Une fois que notre modèle est entrainer et arrive a correctement prédire le prochain feedback, l'agent poura prendre la meilleur action.

# Agent 2
Nous reprenons l'agent 1 et nous allons lui ajouter une fonction 'decide', qui permet de choisir la bonne action a faire en suivant une valence et les prédictions.

In [26]:
class Agent2:
    def __init__(self, model, all_outcomes, all_actions, valance, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valance=valance

    def fit(self, actions:list, outcomes:list, validate_loader=None):
        """
        Fonction d'entrainement de l'agent
        """
        actions = [[self._tokenizer.encode(act)] for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            actions = torch.tensor(actions, dtype=torch.float).to(device)
            outcomes = torch.tensor(outcomes, dtype=torch.long).to(device)
            outcomes = torch.nn.functional.one_hot(outcomes, 
                num_classes=len(self._all_outcomes) # On précise le nombre d'ouctomes possible 
                ).to(torch.float)
            
            data_loader = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(actions, outcomes),
                batch_size=32, shuffle=True
            )

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=10,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass
    
    # Ajout par raport a l'agent 1 bonus
    def decide(self):
        """
        Fonction qui va choisir la meilleur action a faire, dépandament des prédictions du modèles entrainné.
        """
        best_act = self._all_actions[0]
        best_expected_val = -np.inf
        for act in self._all_actions:
            predi = self.predict(act)
            expected_val = self._valance[inter(act, predi)]
            if expected_val > best_expected_val:
                best_act = act
                best_expected_val = expected_val
        self._action = best_act
        return best_act

    def predict(self, action):
        """
        Funciton de prédiction
        """
        action = self._tokenizer.encode(action)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor([action], dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                self.fit(self._history_act, self._history_fb, validate_loader)
            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = self.decide()
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [ ]:
env_test2 = env1()

model_ML = DeepNetwork(hidden_size=[10, 5], input_size=1, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -1,
    inter('b', 'y') : 1
}
agent_test2 = Agent2(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valance=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(20):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [10, 5]
======================= iteration 0 =======================
Action de base : a Prediction: y
action a predi y outcome x
Action choisie : a 0 


======================= iteration 1 =======================
Action: a, Prediction: y, Outcome: x,  Satisfaction: False 
Epoch 1/10, Loss: 0.7937
Epoch 2/10, Loss: 0.6944
Epoch 3/10, Loss: 0.5426
Epoch 4/10, Loss: 0.3785
Epoch 5/10, Loss: 0.2087
Epoch 6/10, Loss: 0.0738
Epoch 7/10, Loss: 0.0136
Epoch 8/10, Loss: 0.0015
Epoch 9/10, Loss: 0.0001
Epoch 10/10, Loss: 0.0000
action a predi x outcome x
Action choisie : a 0 


======================= iteration 2 =======================
Action: a, Prediction: x, Outcome: x,  Satisfaction: True 
action a predi x outcome x
Action choisie : a 0 


======================= iteration 3 =======================
Action: a, Prediction: x, Outcome: x,  Satisfaction: True 
action a predi x outcome x
Action choisie : a 0 


======================= iteration 4 =======================
Action: 

# Résultats
Au début le modèl peut avoir de la chance et correctement prédire une action sur deux corectement. Dans ce cas nous risquons de ne pas tester toutes les actions possible. Si l'agent ne teste pas toutes les actions possible alors il risque de se contenté d'une seul action même si elle ne maximise pas la valence.

## Exemple :
Si le modèl prédit initialement (c'est à dire sans entrainement), a => x et b => x. Et que la valance est celle ci :

```py
valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -10,
    inter('b', 'y') : 10
}
```

Alors le modèl va préférer l'action 'a' indépendament du vrai résultat de de l'action b.

Si l'environement renvois :

- 'a' : 'x'
- 'b' : 'y'

Nous préférions que l'agent chocise l'action 'b'.

## Solution
Nous avons plusieurs solutions possible a ce problème.

### Tester toutes les actions possible
Nous pouvons tester toutes les actions possible pour pouvoir prédire correctement.

### Mécanisme d'ennui
Nous pouvons imaginer que notre agent a 'envie' d'explorer son environement pour s'assurer que ses prédictions sont correct. Nous pouvons mettre en place un cycle qui tout les X temps choisir une autre actions. Nous pouvons pousser ce mécanisme en le lien avec les probabilité, et choisir de tenter des actions pour lequel le modèl n'est pas sûr du résulat.

### Probabilités
Nous pouvons regarder les probablités pour chaques actions. Si pour une action le modèl donne de faible probababilité, alors l'agent peut vouloir s'entrainner sur ces actions. 


# Solution, probabilités

Implémentation de l'agent avec cette notion de probabilité

In [30]:
class Agent2Prob:
    def __init__(self, model, all_outcomes, all_actions, valance, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valance=valance

    def fit(self, actions:list, outcomes:list,nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent
        """
        actions = [[self._tokenizer.encode(act)] for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            actions = torch.tensor(actions, dtype=torch.float).to(device)
            outcomes = torch.tensor(outcomes, dtype=torch.long).to(device)
            outcomes = torch.nn.functional.one_hot(outcomes, 
                num_classes=len(self._all_outcomes) # On précise le nombre d'ouctomes possible 
                ).to(torch.float)
            
            data_loader = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(actions, outcomes),
                batch_size=32, shuffle=True
            )

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        action = self._tokenizer.encode(action)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor([action], dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x
    
    # Ajout par raport a l'agent 1 bonus
    def decide(self):
        """
        Fonction qui choisit l'action a faire en fonction des prédictions \
        du modèles entrainné. Nous renforçons choisisons les actions que \
        ou le modèle n'est pas sûr.
        """
        best_act = self._all_actions[0]
        best_expected_val = -np.inf
        # Vérifie si le modèles est sur de sa prédiction
        for act in self._all_actions:
            probs:torch.Tensor = self.get_prediction(act)
            max_prob = torch.max(probs).item()
            # Formule utiliser : 1 / n + 0.5 / n
            print(f'for action {act} probs {probs} max_prob {max_prob}')
            if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
                return act
            # Si le modèle as une prédiction sur, on regarde sa valance
            predi = self._tokenizer.decode(torch.argmax(probs, dim=0).item())
            expected_val = self._valance[inter(act, predi)]
            if expected_val > best_expected_val:
                best_act = act
                best_expected_val = expected_val
        self._action = best_act
        return best_act

    def predict(self, action):
        """
        Funciton de prédiction
        """
        action = self._tokenizer.encode(action)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor([action], dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = self.decide()
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [31]:
env_test2 = env1()

model_ML = DeepNetwork(hidden_size=[10, 5], input_size=1, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -1,
    inter('b', 'y') : 1
}
agent_test2 = Agent2Prob(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valance=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(20):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [10, 5]
======================= iteration 0 =======================
Action de base : a Prediction: y
action a predi y outcome x
Action choisie : a 0 


======================= iteration 1 =======================
Action: a, Prediction: y, Outcome: x,  Satisfaction: False 
Epoch 1/5, Loss: 0.7538
Epoch 2/5, Loss: 0.5847
Epoch 3/5, Loss: 0.4935
Epoch 4/5, Loss: 0.4183
Epoch 5/5, Loss: 0.3414
for action a probs tensor([0.7727, 0.2273], grad_fn=<SoftmaxBackward0>) max_prob 0.772729218006134
for action b probs tensor([0.7871, 0.2129], grad_fn=<SoftmaxBackward0>) max_prob 0.7870858311653137
action a predi x outcome x
Action choisie : a 0 


======================= iteration 2 =======================
Action: a, Prediction: x, Outcome: x,  Satisfaction: True 
for action a probs tensor([0.7727, 0.2273], grad_fn=<SoftmaxBackward0>) max_prob 0.772729218006134
for action b probs tensor([0.7871, 0.2129], grad_fn=<SoftmaxBackward0>) max_prob 0.7870858311653137
action a predi x outco

# Résultat :
L'agent renforce ca connaicance sur 'a', mais renforce aussi sa __mauvaisse__ connaissance sur 'b'. Il est sûr que 'b' donne 'x' sans avoir tester l'action. Pour palier a ce problème, nous pouvons obliger l'agent a tenter toutes les actions possible.

# Nouvelle agent
Nous allons implémenter deux solutions, celle des probabilité et celle de tester toutes les actions possible.

In [42]:
class Agent2ProbAllTest:
    def __init__(self, model, all_outcomes, all_actions, valance, tokenizer, optimizer=None, loss_func=None):
        """ 
        Création de l'agent.
        
        - self._action : action précédente
        - self._predicted_outcome : prédiction de l'outcome précédent
        """
        self._action = None
        self._predicted_outcome = None
        self._model = model
        self._otimizer = optimizer
        self._loss_func = loss_func
        self._tokenizer:SimpleTokenizerV1 = tokenizer
        self._all_outcomes = all_outcomes
        self._all_actions = all_actions
        self._history_act = []
        self._history_fb = []
        self._valance=valance

    def fit(self, actions:list, outcomes:list,nb_epoch:int= 5, validate_loader=None):
        """
        Fonction d'entrainement de l'agent
        """
        actions = [[self._tokenizer.encode(act)] for act in actions]
        outcomes = self._tokenizer.encode(outcomes)
        if isinstance(self._model, torch.nn.Module):
            self._model.train()
            actions = torch.tensor(actions, dtype=torch.float).to(device)
            outcomes = torch.tensor(outcomes, dtype=torch.long).to(device)
            outcomes = torch.nn.functional.one_hot(outcomes, 
                num_classes=len(self._all_outcomes) # On précise le nombre d'ouctomes possible 
                ).to(torch.float)
            
            data_loader = torch.utils.data.DataLoader(
                torch.utils.data.TensorDataset(actions, outcomes),
                batch_size=32, shuffle=True
            )

            train_with_batch(model=self._model, 
                    train_loader=data_loader,
                    optimizer=self._otimizer,
                    loss_func=self._loss_func,
                    nb_epochs=nb_epoch,
                    validate_loader=validate_loader,
                    print_=True)
        else: # Si le model n'est pas un model pytorch
            raise Exception('Not implemented')
            self._model.fit(action, outcome)
            pass

    def get_prediction(self, action):
        action = self._tokenizer.encode(action)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor([action], dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.nn.functional.softmax(x, dim=0)
        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return x
    
    # Nouvelle fonction qui renvois None si le modèle a tester toutes les actions
    def check_all_actions(self):
        act_to_test = None
        for act in self._all_actions:
            if act not in self._history_act:
                act_to_test = act
                break
        return act_to_test
    
    def decide(self):
        """
        Fonction qui choisit l'action a faire en fonction des prédictions \
        du modèles entrainné. Nous renforçons choisisons les actions que \
        ou le modèle n'est pas sûr.
        """

        act_test = self.check_all_actions()
        if act_test:
            print("i don't know", act_test)
            self._action = act_test
            return act_test

        best_act = self._all_actions[0]
        best_expected_val = -np.inf
        # Vérifie si le modèles est sur de sa prédiction
        for act in self._all_actions:
            probs:torch.Tensor = self.get_prediction(act)
            max_prob = torch.max(probs).item()
            # Formule utiliser : 1 / n + 0.5 / n
            print(f'for action {act} probs {probs} max_prob {max_prob}')
            if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
                print("je ne suis pas sur de ", act)
                self.fit(self._history_act, self._history_fb, validate_loader=None)

            probs:torch.Tensor = self.get_prediction(act)
            if max_prob < 1 / probs.size(dim=0) + 0.5 / probs.size(dim=0):
                print("je n'arrive pas à être sur de ", act)
                return act
            
            # Si le modèle as une prédiction sur, on regarde sa valance
            predi = self._tokenizer.decode(torch.argmax(probs, dim=0).item())
            expected_val = self._valance[inter(act, predi)]
            if expected_val > best_expected_val:
                best_act = act
                best_expected_val = expected_val
        self._action = best_act
        return best_act

    def predict(self, action):
        """
        Funciton de prédiction
        """
        action = self._tokenizer.encode(action)
        if isinstance(self._model, torch.nn.Module):
            self._model.eval() 
            action = torch.tensor([action], dtype=torch.float).to(device)
            x = self._model(action)
            x = torch.argmax(x, dim=0).item()

        else:
            raise Exception('Not implemented')
            x=self._model.predict(action)
        
        return self._tokenizer.decode(x)

    def action(self, outcome, fit=True, validate_loader=None):
        """ 
        Fonction qui choisit l'action a faire en fonction de la dernière \
        intéraction avec l'environnement. \n
        C'est ici que nous allons implémenter un mécanisme de ML \
        pour choisir la prochaine action.

        :param: **outcome** feedback de la dernière intéraction avec l'environnement

        :return: **action** action à effectuer
        """
        if self._action is not None:
            self._history_fb.append(outcome)
            print(f"Action: {self._action}, Prediction: {self._predicted_outcome}, Outcome: {outcome}, " 
                  f"\033[0;31m Satisfaction: {self._predicted_outcome == outcome} \033[0m")
            if self._predicted_outcome != outcome:
                self.fit(self._history_act, self._history_fb, validate_loader=validate_loader)
            # Maintenant nous choisissons la prochaine action en fonction de la valance
            self._action = self.decide()
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
        else:
            self._action = self._all_actions[0]
            self._history_act.append(self._action)
            self._predicted_outcome = self.predict(self._action)
            print(f"Action de base : {self._action} Prediction: {self._predicted_outcome}")
        
        return self._action, self._predicted_outcome

In [43]:
env_test2 = env1()

model_ML = DeepNetwork(hidden_size=[10, 5], input_size=1, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -1,
    inter('b', 'y') : 1
}
agent_test2 = Agent2ProbAllTest(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valance=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(20):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [10, 5]
======================= iteration 0 =======================
Action de base : a Prediction: x
action a predi x outcome x
Action choisie : a 0 


======================= iteration 1 =======================
Action: a, Prediction: x, Outcome: x,  Satisfaction: True 
i don't know b
action b predi x outcome y
Action choisie : b 0 


======================= iteration 2 =======================
Action: b, Prediction: x, Outcome: y,  Satisfaction: False 
Epoch 1/5, Loss: 0.7508
Epoch 2/5, Loss: 0.6654
Epoch 3/5, Loss: 0.6310
Epoch 4/5, Loss: 0.6550
Epoch 5/5, Loss: 0.6118
for action a probs tensor([0.5552, 0.4448], grad_fn=<SoftmaxBackward0>) max_prob 0.5551952123641968
je ne suis pas sur de  a
Epoch 1/5, Loss: 0.6128
Epoch 2/5, Loss: 0.5934
Epoch 3/5, Loss: 0.5524
Epoch 4/5, Loss: 0.5158
Epoch 5/5, Loss: 0.4858
je n'arrive pas à être sur de  a
action a predi x outcome x
Action choisie : a 0 


======================= iteration 3 =======================
Action: a, Predi

# Résultat
L'agent arrive a tester toutes les actions et renforce ses connaissances sur les actions dont il n'est sûr. 

In [44]:
env_test2 = env6Str()

model_ML = DeepNetwork(hidden_size=[10, 5], input_size=1, output_size=2)
optimizer = torch.optim.Adam(model_ML.parameters(), lr=1e-1, weight_decay=1e-2)
loss_func = nn.CrossEntropyLoss()
tokenizer = SimpleTokenizerV1(create_dico_numerate_word(env_test2.get_outcomes() + env_test2.get_actions()))

valence = {
    inter('a', 'x') : -1,
    inter('a', 'y') : 1,
    inter('b', 'x') : -1,
    inter('b', 'y') : 1
}
agent_test2 = Agent2ProbAllTest(
    model=model_ML,
    all_outcomes= env_test2.get_outcomes(),
    all_actions= env_test2.get_actions(),
    valance=valence,
    tokenizer=tokenizer,
    optimizer=optimizer,
    loss_func=loss_func)

history_good = []
pourcent_by_10 = []
outcome = None
for i in range(20):
    print(f"=======================\033[0;32m iteration {i} \033[0m=======================")
    action, predi = agent_test2.action(outcome, False)
    outcome = env_test2.outcome(action)
    history_good.append(outcome == predi)
    pourcent_by_10.append(sum(history_good[-10:]) * 10 if len(history_good) >= 10 else 0)
    print(f'action {action} predi {predi} outcome {outcome}')
    print(f"Action choisie : {action} \033[0;34m{pourcent_by_10[-1]} \033[0m")
    print("\n")
    

liste hidden init [10, 5]
======================= iteration 0 =======================
Action de base : a Prediction: x
action a predi x outcome y
Action choisie : a 0 


======================= iteration 1 =======================
Action: a, Prediction: x, Outcome: y,  Satisfaction: False 
Epoch 1/5, Loss: 0.8688
Epoch 2/5, Loss: 0.6334
Epoch 3/5, Loss: 0.4280
Epoch 4/5, Loss: 0.1810
Epoch 5/5, Loss: 0.0330
i don't know b
action b predi y outcome x
Action choisie : b 0 


======================= iteration 2 =======================
Action: b, Prediction: y, Outcome: x,  Satisfaction: False 
Epoch 1/5, Loss: 3.7932
Epoch 2/5, Loss: 2.8725
Epoch 3/5, Loss: 1.8562
Epoch 4/5, Loss: 1.1559
Epoch 5/5, Loss: 0.8428
for action a probs tensor([0.3268, 0.6732], grad_fn=<SoftmaxBackward0>) max_prob 0.6732020378112793
je ne suis pas sur de  a
Epoch 1/5, Loss: 0.7571
Epoch 2/5, Loss: 0.7443
Epoch 3/5, Loss: 0.7324
Epoch 4/5, Loss: 0.7217
Epoch 5/5, Loss: 0.7127
je n'arrive pas à être sur de  a
action